In [ ]:
#import des librairies
from collatex import *
from lxml import etree
import os
from xml.etree import ElementTree as et
import json

In [ ]:
#on a deux fichiers, Apar.xml et Bpar.xml : on va les parser

A1 = etree.parse('textes/Apar.xml')
B1 = etree.parse('textes/Bpar.xml')

In [ ]:
#parser permet de récupérer les valeurs des différents noeuds, par exemple
A1.xpath("/text/p/@n")

In [ ]:
#défintion de la fonction qui permet la transformation vers du json, puisque collatex doit prendre du json
#on utilise une feuille de transformation
def XMLtoJson(id,xmlInput):
    #défition de witness comme un dictionnaire
    witness = {}
    #on donne la première valeur de l'id
    witness['id'] = id
    #on transforme le document XML
    monXSL = etree.XML('''
<xsl:stylesheet xmlns:xsl="http://www.w3.org/1999/XSL/Transform"
    xmlns:xs="http://www.w3.org/2001/XMLSchema"
    exclude-result-prefixes="xs"
    version="1.0">

    <xsl:output method="text"/>

    <xsl:template match="/">
        <xsl:apply-templates/>
    </xsl:template>

    <xsl:template match="w">
        <xsl:text>{"text": "</xsl:text>
        <xsl:apply-templates/>
        <xsl:text>", "i": "</xsl:text>
        <xsl:value-of select="@xml:id"/>
        <xsl:text>", "t": "</xsl:text>
        <xsl:value-of select="@lemma"/>
        <xsl:text>", "pos": "</xsl:text>
        <xsl:value-of select="@pos"/>
        <xsl:text>"}</xsl:text>
        <xsl:if test="following::w">
            <xsl:text>, </xsl:text>
        </xsl:if>
    </xsl:template>
</xsl:stylesheet>
    ''')
    #on applique l'XSL
    monXSL = etree.XSLT(monXSL)
    #on charge le résultat de l'XSL dans la valeur de l'entrée du dicitonnaire "tokens"
    witness['tokens'] = json.loads( '[' +str(monXSL(xmlInput)) +']')
    return witness

In [ ]:
#définition de la fonction notre export

def notre_export_xml(collation):
    readings = []
    for column in table.columns:
        #création élément app
        app = et.Element('app')
        for key, value in sorted(column.tokens_per_witness.items()):
            child = et.Element('rdg')
            child.attrib['wit'] = "#" + key
            #on créer les attributs pour garder nos valeurs
            for item in value:
                child.attrib['xml:id'] = item.token_data["i"]
                child.attrib['lemma']= item.token_data["t"]
                child.attrib['pos'] = item.token_data["pos"]
            child.text = "".join(str(item.token_data["text"]) for item in value)
            app.append(child)
        result = et.tostring(app, encoding="unicode")
        readings.append(result)
    return "".join(readings)

In [ ]:
#création d'une liste vide dans laquelle on va stocker nos valeurs
listeDesP = []

#création d'une boucle
for par in A1.xpath("/text/p"):
    #on récupère le numéro de paragraphe
    i = par.get('n')
    # on vérifie que tout se passe bien
    print(i)
    #A = le paragraphe en question
    A = par
    print(A)
    #on récupère aussi le paragraphe qui correspond en Ez (B)  
    for p in B1.xpath("/text/p[@n='"+i+"']"):
        #B = le paragraphe qui porte le même numéro que en A
        B = p
        print(B)
        
    #définition des témoins
        #le dictionnaire niveau général
        json_input = {}
        #la première entrée qui contient les witnesses
        json_input['witnesses'] = []
        #application de la fonction de transformation sur A puis sur B
        json_input['witnesses'].append(XMLtoJson('A',A))
        json_input['witnesses'].append(XMLtoJson('B',B))
        print(json_input)
    #on a bien à chaque fois des json qui se sont créés
    
        #création de l'objet collation
        json_collation = Collation()
        #pour chaque témoin dans le json
        for witness in json_input["witnesses"]:
            #on utilise la fonction add_witness de la classe Collation
            json_collation.add_witness(witness)
        #changement de nom    
        collation = json_collation
        print(collation)
        #on collationne avec une sortie table
        table = collate(collation, output="table",segmentation=False)
        #on utilise la fonction export
        result = notre_export_xml(table)
        #on crée un élément p pour chacune des collations réalisées
        doc = '<p n="' + i +'">' + result + '</p>'
        #on ajoute à la liste l'ensemble créé
        listeDesP.append(doc)

In [ ]:
listeDesP

In [ ]:
#on crée un objet dans lequel on place notre liste, à l'intérieur d'une balise racine <text>
docCollPar = "<text>" + ''.join(listeDesP) + "</text>"

docCollPar

In [ ]:
#on écrit le résultat (une simple chaîne de caractères) dans notre document XML
with open("resultat_collation_paragraphes.xml", "w") as text_file:
    text_file.write(docCollPar)   